In [7]:
%matplotlib inline

import pandas,os

hdf_files = lambda fpth : [
                            './data/{fpth}/{file_}'.format(
                                fpth=fpth,
                               file_=file_ )
                               for file_ in os.listdir('./data/{fpth}/'.format(fpth=fpth))
                                if file_.split('.')[-1] == 'h5']


In [8]:
traffic_hdfs = pandas.concat(
                  map(
        lambda file_ : pandas.read_hdf(
                file_,
                  key='table'),
                   hdf_files('traffic')))

In [12]:
traffic_hdfs.shape

(18903, 4)

In [15]:
traffic_hdfs.groupby(
    ['pickup_latitude','pickup_longitude','hour']
).sum().describe()

,key
count,13272.000000
mean,150.693189
std,645.912429
min,1.000000
25%,1.000000
50%,2.000000
75%,7.000000
max,8830.000000


In [ ]:
pool.close()